
<div class="text-center">
<h1 class="display-1" >Data Mining </h1>
<h3> Homework 2</h3>
<h3> Di Santo Nicola - 1853049</h3>

</div>

# EX. 3

## Utilities From Ex2

In [223]:
import re, hashlib, math, time
from random import randint, seed
seed(16)


class hashFamily:
    def __init__(self, i):
        self.resultSize = 8 # how many bytes we want back
        self.maxLen = 20 # how long can our i be (in decimal)
        self.salt = str(i).zfill(self.maxLen)[-self.maxLen:]
        self.id = i
        
    def get_hash_value(self, el_to_hash):
        return int(hashlib.sha1(str(el_to_hash).encode('utf-8') + self.salt.encode('utf-8')).hexdigest()[-self.resultSize:], 16)
    

class shingler:
    def __init__(self, k):
        
        if k > 0:
            self.k = int(k)
        else:
            self.k = 10
        
    #inner class utility
    def process_doc(self, document):
        return re.sub("( )+|(\n)+"," ",document).lower()
    
    def get_shingles(self, document):
        shingles = set()
        document= self.process_doc(document)
        for i in range(0, len(document)-self.k+1 ):
            shingles.add(document[i:i+self.k])
        return shingles
    
    def get_k(self):
        return self.k
    
    #return sorted hash
    def get_hashed_shingles(self, shingles_set):
        hash_function = hashFamily(0)
        return sorted( {hash_function.get_hash_value(s) for s in shingles_set} )

## Init

In [224]:
import findspark
findspark.init()

In [225]:
from pyspark.sql import SparkSession

file_path = "../Ex2/dataset_rent_rome_kijiji.tsv"  
spark = SparkSession.builder.getOrCreate()
segments = spark.read.csv( path=file_path, header=True, sep='\t' )

#segments.persist() # to avoid lazy behaviour and store dataset in memory
segments.show() # data preview


+--------------------+--------------------+--------------------+------------------+-------------------+--------------------+
|               Title|   Short Description|            Location|      Price (Euro)|          Timestamp|             Url Adv|
+--------------------+--------------------+--------------------+------------------+-------------------+--------------------+
|Studio accessoria...|Affitto studio a ...|               Roma |              450 | 12 ottobre, 11:32 |https://www.kijij...|
|Negozio 169Mq per...|Privato affitta n...|Prenestino / Casi...|            1.700 | 12 ottobre, 08:45 |https://www.kijij...|
|Negozio in tiburt...|Negozio c/1 roma ...|Tiburtino / Colla...|            6.000 | 17 October, 21:20 |https://www.kijij...|
|Studio medico via...|Studio medico avv...|Trieste / Somalia...|              200 | 17 October, 20:22 |https://www.kijij...|
|Cerco: Appartamen...|Donna lavoratrice...|               Roma |Contatta l'utente | 17 October, 19:39 |https://www.kijij...|


In [226]:
from pyspark.sql.functions import monotonically_increasing_id 

dataset = segments.select('Title','Short Description').withColumn("doc_id", monotonically_increasing_id())
dataset.show()
dataset.persist()


+--------------------+--------------------+------+
|               Title|   Short Description|doc_id|
+--------------------+--------------------+------+
|Studio accessoria...|Affitto studio a ...|     0|
|Negozio 169Mq per...|Privato affitta n...|     1|
|Negozio in tiburt...|Negozio c/1 roma ...|     2|
|Studio medico via...|Studio medico avv...|     3|
|Cerco: Appartamen...|Donna lavoratrice...|     4|
|Elegante studio m...|Studio medico con...|     5|
|Ufficio su strada...|A pochi metri da ...|     6|
|  Camera in affitto |Camera per studen...|     7|
|Magazzino Via Giu...|Affittasi Locale ...|     8|
|Negozio + Piazzal...|Privato AFFITTA S...|     9|
|Locale uso studio...|Affittasi N.1 loc...|    10|
|Sala per Feste pe...|Spazio modulabile...|    11|
|Sala per feste a ...|Affitto locale a ...|    12|
|  Box insonorizzato |Box con soppalco ...|    13|
|Stanze per Medici...|Affittasi 1 o 2 S...|    14|
|Appartamento in m...|Appartamento in v...|    15|
|Location per fest...|Per una f

DataFrame[Title: string, Short Description: string, doc_id: bigint]

## Shingling
<p>For shinglings we cannot parallelize on a single document but among all the documents of the dataset. The Map function for shingling will then only distribute documents among worker nodes in the network producing (doc id,Shingles set, hash_id) pairs. The reduce function is useless for now. This step can be thought as a simple transformation step on the Spark dataframe, in this way we are chunking the characteristc matrix by column.</p> <p>Since we are not on a cluster but on a single local machine, the result will be the same as using the shingler class only, however I slightly modify the algorithm used in Ex2 to use map-reduce paradigm.</p>
<p>The hash_id value is used to simplify the minwise hashing map step below: the hash function to use is already encoded into distributed data and next map step will be easier. With this approach we pay in terms of pair generated as output. However, since the dataset size is small it is still possible to consider this approach.</p>

In [227]:
def shingling_map(row):
    out = [] 
    sh_instance = shingler(10)
    hashed_shingles = sh_instance.get_hashed_shingles( sh_instance.get_shingles( row['Title']+" "+row['Short Description'] ) ) 
    signature_size = 50
    for i in range(0,signature_size): #signature size
        out.append( (row['doc_id'], hashed_shingles, i ) ) 
    #return an iterator to use flatMap => produce more than one key-value pair as output (namely one per hash function)
    return iter(out)

    
#Use rdd.collect() to get all data from workers to driver. In the specific case it returns a list of [(doc_id, shingle_set),...] where shingle_set = [sh1,sh2,...sh_n]
docId_shingleset_hfunc = dataset.rdd.flatMap(shingling_map)
#doc_id_shingle_set_pair.toDF().show() to see results

## Minwise Hashing
For minhashing, since the characteristic matrix is chunked by column, we can easly design a map-reduce task to produce the signature matrix: 
<ul>
    <li>Map: take as input $(doc\_id, shingle\_set,h_i)$ tuple with $h_i$ an hash function from hash family defined above and produce the minhash value of the set for that given hash function: $(doc\_id,  min\_hash)$</li>
    <li>Reduce: the reduce step is simplified to a simple, built in, <i>groupByKey()</i> action and returns a column of the signature matrix: $(doc\_id, minhash\_signature)$</li>
</ul>

In [228]:
def minhash_map(docId_ShingleSet_hFunct):
    doc_id = docId_ShingleSet_hFunct[0]
    shingles = docId_ShingleSet_hFunct[1]
    hash_f = hashFamily( docId_ShingleSet_hFunct[2] )
    min_h = math.inf
    for el in shingles:
        hash_value = hash_f.get_hash_value(el)
        if hash_value < min_h:
            min_h = hash_value
            
    return (doc_id, min_h )


# as Reduce step we use the built in groupByKey() since no extra operation is needed.
sig_matrix = docId_shingleset_hfunc.map(minhash_map).groupByKey().map(lambda x : (x[0], list(x[1])))
sig_matrix.persist() # to do not compute it every time since its an expensive computation
#examples of content are:
dummy = ''' 

sig_matrix.toDF().show()
+---+--------------------+
| _1|                  _2|
+---+--------------------+
|  0|[4781236, 3571529...|
|  1|[2130985, 1259985...|
|  2|[40365747, 857165...|
|  3|[45686806, 309643...|
|  4|[8369628, 2430029...|
|  5|[39442189, 141331...|
|  6|[2999960, 2797603...|
|  7|[30153137, 360865...|
|  8|[49068494, 273188...|
|  9|[1344538, 5459810...|
| 10|[7879527, 7920209...|
| 11|[7700705, 6769721...|
| 12|[272635, 1963659,...|
| 13|[2999960, 1486707...|
| 14|[1171357, 5761465...|
| 15|[21922776, 310437...|
| 16|[1813937, 5020567...|
| 17|[76445429, 575841...|
| 18|[3295091, 2904792...|
| 19|[3295091, 2904792...|
+---+--------------------+



sig_matrix.take(1)
[(0,
  [4781236,
   35715291,
   567066,
   390697,
   5723987,
   2079438,
   6462491,
   7336631,
   1957111,
   39763103])]
'''


## Locality Sensitive Hashing
<p>In this step we are going to: 
<ol>
    <li>Divide Signature Matrix into bands</li>
    <li>Hash the bands to get candidates pairs</li>
    <li>Check if candidates are actually similar</li>
</ol>
The first two step are computed among worker nodes while the last one, as well as bruteforce comparisons, is computed on master node since it requires lookup to signature martix. Lookup are not allowed on rdd datastructure from worker nodes.</p>

In [229]:

# IMPLEMENTING POINT 1. AND 2. ABOVE WITH A SINGLE MAP-REDUCE STEP
# with this function we split signature into equal subset, assign this subset to a band, hash the subset to get its bucket nr.
# the return value will be a key-value pair with key = (band_id, bucket) and value = doc_id
# Note: The key include the band-id because we used a single hash function to produce buckets for different bands 
#       and its fine as long as we produce different buckets for the bands (from chap 3.4 of the book)

# The reduce step will then be a simple groupByKey() and will give us key = (band_id, bucket), value = list of candidates
# Note: an additional map is performed to transform iterable object to list 
def map_buckets(row):
    
    band_nr = 5
    row_nr = 10
    doc_id = row[0]
    doc_sign = row[1]
    hash_funct = hashFamily(1)
    out = []
    
    for i in range(0,band_nr):
        band_id = i
        idx = i*row_nr   
        set_col = ' '.join(str(x) for x in doc_sign[idx:idx+row_nr])
        bucket = hash_funct.get_hash_value(set_col)
        out.append( ( (band_id, bucket), doc_id)  )
    
    return iter(out) #since we are going to return multiple tuple we call flatMap() and return an iterator on those tuple
    
candidates = sig_matrix.flatMap(map_buckets).groupByKey().map(lambda x : (x[0], list(x[1]))) #the map() is used to convert iterator to a list

In [230]:
#example of data obtained from the above map-reduce-map step
dummy_candidates_ex = ''' candidates.take(1)

[((0, 2699225922),
  [0,
   22,
   44,
   66,
   88,
   154,
   176,
   220,
   330,
   352,
   440,
   603,
   647,
   669,
   735,
   757,
   779,
   801,
   867,
   889,
   933,
   1043,
   1087,
   1109,
   1153,
   1219,
   1241,
   1263,
   1373,
   1417,
   1461,
   1505,
   1615,
   1637,
   1703,
   1725,
   1813,
   1857,
   1901,
   1967,
   1989,
   2011,
   2033,
   2077,
   2099,
   2143,
   2187,
   2297,
   2319,
   2363,
   2451,
   2517,
   2561])]
   
   candidates.toDF().show()
    +---------------+--------------------+
|             _1|                  _2|
+---------------+--------------------+
|[0, 2699225922]|[0, 22, 44, 66, 8...|
| [1, 773591669]|[0, 22, 44, 66, 8...|
|[2, 3964590397]|[0, 22, 44, 66, 8...|
|[3, 2852783413]|[0, 22, 44, 66, 8...|
|[4, 1818672075]|[0, 22, 44, 66, 8...|
|[0, 1417325792]|[1, 23, 45, 67, 8...|
| [1, 485583475]|[1, 23, 45, 67, 8...|
| [2, 823708336]|[1, 23, 45, 67, 8...|
|[3, 3657351485]|[1, 23, 45, 67, 8...|
|[4, 1476921203]|[1, 23, 45, 67, 8...|
|[0, 1902259772]|           [2, 1257]|
| [1, 869183940]|                 [2]|
|[2, 3109000709]|           [2, 1257]|
| [3, 319528939]|                 [2]|
|[4, 2004917486]|                 [2]|
|[0, 2096920849]|                 [3]|
|[1, 2844631318]|                 [3]|
|[2, 3397623607]|                 [3]|
|[3, 1948736288]|                 [3]|
|[4, 3121195643]|                 [3]|
+---------------+--------------------+
'''


In [238]:
## This part of the algorithm must be performed at driver since lookup into signature matrix are required and 
## spark does not allow lookup from worker nodes. 

similar_pairs = set()
sig_df = sig_matrix
sig_df.persist() #store it in memory to avoid lazy behaviour
sig_df = sig_df.collect() #collect all the distributed record to speedup computation
start_time = time.time()

for candidates_list in candidates.map(lambda x: x[1]).collect(): #collecting all candidate lists from worker nodes
    
    candidates_nr = len(candidates_list)
    
    for i in range(0,candidates_nr-1):
        for j in range(i+1,candidates_nr):
            
            doc_id_1 = candidates_list[i]
            doc_id_2 = candidates_list[j]
            
            docId_Sig_1 = sig_df[doc_id_1]
            docId_Sig_2 = sig_df[doc_id_2]
            if docId_Sig_1[0] != doc_id_1:
                raise Exception("DocId is %d while retrived %d from signature matrix."%(doc_id_1,docId_Sig_1[0]))
            if docId_Sig_2[0] != doc_id_2:
                raise Exception("DocId is %d while retrived %d from signature matrix."%(doc_id_1,docId_Sig_2[0])) 
                
            sig_1 = set(docId_Sig_1[1]) #transform signature into a set
            sig_2 = set(docId_Sig_2[1])
            #sig_1 = sig_df.filter("_1 =="+" "+ str(doc_id_1) ).select('_2') #get signature of doc 1
            #sig_2 = sig_df.filter("_1 =="+" "+ str(doc_id_2) ).select('_2') #get signature of doc 2
            
            #sig_2 = set(sig_2.collect()[0]['_2'])
            js = len(sig_1.intersection(sig_2) ) / len(sig_1.union(sig_2) ) #Compute Jaccard'Similarity
            if js >= 0.8:
                pair = tuple(sorted((doc_id_1,doc_id_2) ))
                similar_pairs.add(  pair   ) 

end_time = time.time()  

lsh_time = end_time - start_time

print("FOUND %d SIMILAR PAIRS" %len(similar_pairs))

#sim_pairs = candidates.flatMap(get_similar_items).groupByKey().map(lambda x : (x[0], set(serted(x[1]))))

FOUND 10322 SIMILAR PAIRS


Bruteforce search:

In [239]:
DocID_Shingles=docId_shingleset_hfunc.map(lambda x: (x[0],x[1]) ).reduceByKey(lambda x,y: x).collect()
#DocID_Shingles.persist()
doc_nr = len(DocID_Shingles)
similar_pairs_bf = set()

start_time = time.time()

for i in range(0,doc_nr-1):
    for j in range(i,doc_nr):
        doc_id_1 = DocID_Shingles[i][0] #DocID_Shingles[i] is  tuple (doc_id, shingle_set)
        doc_id_2 = DocID_Shingles[j][0]
        
        sig_1 = set(DocID_Shingles[i][1]) #transform shingle_set into a python set data structure
        sig_2 = set(DocID_Shingles[j][1])
        js = len(sig_1.intersection(sig_2) ) / len(sig_1.union(sig_2) ) #Compute Jaccard'Similarity
        if js >= 0.8:
            pair = tuple(sorted((doc_id_1,doc_id_2) ))
            similar_pairs_bf.add(  pair   ) 
            
end_time = time.time()
    
bf_time = end_time - start_time 

print("FOUND %d SIMILAR PAIRS WITH BRUTEFORCE" %len(similar_pairs_bf))

FOUND 12986 SIMILAR PAIRS WITH BRUTEFORCE


## Stats

In [246]:
print("EXECUTION REPORT")
print("LSH\n%d\tSIMILAR ITEMS\n%.2f\tSECONDS\n"%(len(similar_pairs), lsh_time))
print("BRUTEFORCE\n%d\tSIMILAR ITEMS\n%.2f\tSECONDS\n"%(len(similar_pairs_bf), bf_time))
print("%d SIMILAR PAIR DISCOVERED WITH BRUTEFORCE AND LSH\n"%( len(similar_pairs.intersection(similar_pairs_bf)) ))

print("%d NUMBER OF FALSE POSITIVE SIMILARITIES WITH LSH\n"%(len(similar_pairs) - len(similar_pairs.intersection(similar_pairs_bf)) ) )

print("%d NON DETECTED SIMILARITIES BY LSH"%(len(similar_pairs_bf) - len(similar_pairs.intersection(similar_pairs_bf))) )

EXECUTION REPORT
LSH
10319	SIMILAR ITEMS
1.24	SECONDS

BRUTEFORCE
12986	SIMILAR ITEMS
336.77	SECONDS

10318 SIMILAR PAIR DISCOVERED WITH BRUTEFORCE AND LSH

1 NUMBER OF FALSE POSITIVE SIMILARITIES WITH LSH

2668 NON DETECTED SIMILARITIES BY LSH


## Example
From LSH similar items

In [244]:
for i in range(0,3):
    pair = similar_pairs.pop()
    doc1 = pair[0]
    doc2 = pair[1]
    print("Doc: "+str(i))
    dataset.where('doc_id == '+str(doc1)).show()
    print("Similar Doc:")
    dataset.where('doc_id == '+str(doc2)).show()
    print("\n\n")

Doc: 0
+--------------------+--------------------+------+
|               Title|   Short Description|doc_id|
+--------------------+--------------------+------+
|Avviato studio ps...|Avviato studio di...|   506|
+--------------------+--------------------+------+

Similar Doc:
+--------------------+--------------------+------+
|               Title|   Short Description|doc_id|
+--------------------+--------------------+------+
|Avviato studio ps...|Avviato studio di...|   568|
+--------------------+--------------------+------+




Doc: 1
+--------------------+--------------------+------+
|               Title|   Short Description|doc_id|
+--------------------+--------------------+------+
|Negozio 169Mq per...|Privato affitta n...|    89|
+--------------------+--------------------+------+

Similar Doc:
+--------------------+--------------------+------+
|               Title|   Short Description|doc_id|
+--------------------+--------------------+------+
|Negozio 169Mq per...|Privato affitt